Old cell
```
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

# Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode

## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

# Install TorchAudio
!pip install torchaudio>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html
```


In [1]:
import nemo.collections.asr as nemo_asr
nemo_asr.models.EncDecSpeakerLabelModel.list_available_models()

[PretrainedModelInfo(
 	pretrained_model_name=speakerverification_speakernet,
 	description=For details about this model, please visit https://ngc.nvidia.com/catalog/models/nvidia:nemo:speakerverification_speakernet,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/speakerverification_speakernet/versions/1.16.0/files/speakerverification_speakernet.nemo
 ),
 PretrainedModelInfo(
 	pretrained_model_name=ecapa_tdnn,
 	description=For details about this model, please visit https://ngc.nvidia.com/catalog/models/nvidia:nemo:ecapa_tdnn,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/ecapa_tdnn/versions/1.16.0/files/ecapa_tdnn.nemo
 ),
 PretrainedModelInfo(
 	pretrained_model_name=titanet_large,
 	description=For details about this model, please visit https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/titanet_large,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/titanet_large/versions/v1/files/titanet-l.nemo
 ),
 PretrainedModelInfo(
 	pretrained_

In [2]:
# new cell
BRANCH = 'main'

# from https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/asr/speaker_recognition/intro.html

model_name = 'ecapa_tdnn'

# **SPEAKER RECOGNITION** 
Speaker Recognition (SR) is a broad research area that solves two major tasks: speaker identification (who is speaking?) and
speaker verification (is the speaker who they claim to be?). In this work, we focus on text-independent speaker recognition when the identity of the speaker is based on how the speech is spoken,
not necessarily in what is being said. Typically such SR systems operate on unconstrained speech utterances,
which are converted into fixed-length vectors, called speaker embeddings. Speaker embeddings are also used in
automatic speech recognition (ASR) and speech synthesis.

In this tutorial, we shall first train these embeddings on speaker-related datasets, and then get speaker embeddings from a pretrained network for a new dataset. Since Google Colab has very slow read-write speeds, I'll be demonstrating this tutorial using [an4](http://www.speech.cs.cmu.edu/databases/an4/). 

Instead, if you'd like to try on a bigger dataset like [hi-mia](https://arxiv.org/abs/1912.01231) use the [get_hi-mia-data.py](https://github.com/NVIDIA/NeMo/tree/main/scripts/dataset_processing/speaker_tasks/get_hi-mia_data.py) script to download the necessary files, extract them, and resample to 16Khz if any of these samples are not at 16Khz. 

In [3]:
import os
NEMO_ROOT = os.getcwd()
print(NEMO_ROOT)
import glob
import subprocess
import tarfile
import wget

data_dir = os.path.join(NEMO_ROOT,'data')
os.makedirs(data_dir, exist_ok=True)

# Download the dataset. This will take a few moments...
print("******")
if not os.path.exists(data_dir + '/an4_sphere.tar.gz'):
    an4_url = 'https://dldata-public.s3.us-east-2.amazonaws.com/an4_sphere.tar.gz'
    an4_path = wget.download(an4_url, data_dir)
    print(f"Dataset downloaded at: {an4_path}")
else:
    print("Tarfile already exists.")
    an4_path = data_dir + '/an4_sphere.tar.gz'

# Untar and convert .sph to .wav (using sox)
tar = tarfile.open(an4_path)
tar.extractall(path=data_dir)

print("Converting .sph to .wav...")
sph_list = glob.glob(data_dir + '/an4/**/*.sph', recursive=True)
for sph_path in sph_list:
    wav_path = sph_path[:-4] + '.wav'
    cmd = ["sox", sph_path, wav_path]
    subprocess.run(cmd)
print("Finished conversion.\n******")

/home/eyal/git/hakol/notebooks
******
Tarfile already exists.
Converting .sph to .wav...
Finished conversion.
******


Since an4 is not designed for speaker recognition, this facilitates the opportunity to demonstrate how you can generate manifest files that are necessary for training. These methods can be applied to any dataset to get similar training manifest files. 

First, create a list file which has all the wav files with absolute paths for each of the train, dev, and test set. This can be easily done by the `find` bash command

In [4]:
!find {data_dir}/an4/wav/an4_clstk  -iname "*.wav" > data/an4/wav/an4_clstk/train_all.txt

Let's look at the first 3 lines of filelist text file for train.

In [5]:
!head -n 3 {data_dir}/an4/wav/an4_clstk/train_all.txt

/home/eyal/git/hakol/notebooks/data/an4/wav/an4_clstk/mcen/an126-mcen-b.wav
/home/eyal/git/hakol/notebooks/data/an4/wav/an4_clstk/mcen/cen6-mcen-b.wav
/home/eyal/git/hakol/notebooks/data/an4/wav/an4_clstk/mcen/an130-mcen-b.wav


Since we created the list text file for the train, we use `filelist_to_manifest.py` to convert this text file to a manifest file and then optionally split the files to train \& dev for evaluating the models during training by using the `--split` flag. We wouldn't be needing the `--split` option for the test folder. 
Accordingly please mention the `id` number, which is the field num separated by `/` to be considered as the speaker label 

After the download and conversion, your `data` folder should contain directories with manifest files as:

* `data/<path>/train.json`
* `data/<path>/dev.json` 
* `data/<path>/train_all.json` 

Each line in the manifest file describes a training sample - `audio_filepath` contains the path to the wav file, `duration` it's duration in seconds, and `label` is the speaker class label:

`{"audio_filepath": "<absolute path to dataset>data/an4/wav/an4test_clstk/menk/cen4-menk-b.wav", "duration": 3.9, "label": "menk"}` 

In [6]:
if not os.path.exists('scripts'):
  print("Downloading necessary scripts")
  !mkdir -p scripts/speaker_tasks
  !wget -P scripts/speaker_tasks/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/speaker_tasks/filelist_to_manifest.py
!python {NEMO_ROOT}/scripts/speaker_tasks/filelist_to_manifest.py --filelist {data_dir}/an4/wav/an4_clstk/train_all.txt --id -2 --out {data_dir}/an4/wav/an4_clstk/all_manifest.json --split

100%|███████████████████████████████████████| 948/948 [00:00<00:00, 3892.72it/s]
wrote /home/eyal/git/hakol/notebooks/data/an4/wav/an4_clstk/train_all_manifest.json
100%|██████████████████████████████████████| 948/948 [00:00<00:00, 89839.36it/s]
wrote /home/eyal/git/hakol/notebooks/data/an4/wav/an4_clstk/all_manifest.json
number of train samples after split:  853
wrote /home/eyal/git/hakol/notebooks/data/an4/wav/an4_clstk/train.json
wrote /home/eyal/git/hakol/notebooks/data/an4/wav/an4_clstk/dev.json


Generate the list text file for the test folder and then convert it to a manifest.

In [7]:
!find {data_dir}/an4/wav/an4test_clstk  -iname "*.wav" > {data_dir}/an4/wav/an4test_clstk/test_all.txt
!python {NEMO_ROOT}/scripts/speaker_tasks/filelist_to_manifest.py --filelist {data_dir}/an4/wav/an4test_clstk/test_all.txt --id -2 --out {data_dir}/an4/wav/an4test_clstk/test.json

100%|████████████████████████████████████████| 130/130 [00:00<00:00, 346.95it/s]
wrote /home/eyal/git/hakol/notebooks/data/an4/wav/an4test_clstk/test_all_manifest.json
100%|██████████████████████████████████████| 130/130 [00:00<00:00, 32040.16it/s]
wrote /home/eyal/git/hakol/notebooks/data/an4/wav/an4test_clstk/test.json


## Path to manifest files


In [8]:
train_manifest = os.path.join(data_dir,'an4/wav/an4_clstk/train.json')
validation_manifest = os.path.join(data_dir,'an4/wav/an4_clstk/dev.json')
test_manifest = os.path.join(data_dir,'an4/wav/an4_clstk/dev.json')

As the goal of most speaker-related systems is to get good speaker level embeddings that could help distinguish from
other speakers, we shall first train these embeddings in an end-to-end
manner optimizing the [TitaNet](https://arxiv.org/pdf/2110.04410.pdf) model.
We modify the decoder to get these fixed-size embeddings irrespective of the length of the input audio.

# Training
Import necessary packages

Note: All the following steps are just for explanation of each section, but one can use the provided [training script](https://github.com/NVIDIA/NeMo/blob/main/examples/speaker_tasks/recognition/speaker_reco.py) to launch training in the command line.

In [9]:
import nemo
# NeMo's ASR collection - This collection contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
from omegaconf import OmegaConf

## Model Configuration 
The TitaNet model is defined in a config file which declares multiple important sections.

They are:

1) model: All arguments that will relate to the Model - preprocessors, encoder, decoder, optimizer and schedulers, datasets, and any other related information

2) trainer: Any argument to be passed to PyTorch Lightning

In [10]:
# This line will print the entire config of sample TitaNet model
os.makedirs('conf', exist_ok=True)
url = f"https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/speaker_tasks/recognition/conf/{model_name}.yaml"
wget.download(url, out=f"./conf/{model_name}.yaml")

MODEL_CONFIG = os.path.join(NEMO_ROOT,f'conf/{model_name}.yaml')
config = OmegaConf.load(MODEL_CONFIG)
print(OmegaConf.to_yaml(config))

name: ECAPA_TDNN
model:
  sample_rate: 16000
  train_ds:
    manifest_filepath: ???
    sample_rate: ${model.sample_rate}
    labels: null
    batch_size: 64
    shuffle: true
    augmentor:
      noise:
        manifest_path: null
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: ${model.sample_rate}
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
  validation_ds:
    manifest_filepath: ???
    sample_rate: ${model.sample_rate}
    labels: null
    batch_size: 128
    shuffle: false
  preprocessor:
    _target_: nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor
    normalize: per_feature
    window_size: 0.025
    sample_rate: ${model.sample_rate}
    window_stride: 0.01
    window: hann
    features: 80
    n_fft: 512
    frame_splicing: 1
    dither: 1.0e-05
    stft_conv: false
  spec_augment:
    _target_: nemo.collections.asr.modules.SpectrogramAugmentation
    freq_m

## Setting up the datasets within the config
If you'll notice, there are a few config dictionaries called train_ds, validation_ds and test_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

In [11]:
print(OmegaConf.to_yaml(config.model.train_ds))
print(OmegaConf.to_yaml(config.model.validation_ds))

manifest_filepath: ???
sample_rate: ${model.sample_rate}
labels: null
batch_size: 64
shuffle: true
augmentor:
  noise:
    manifest_path: null
    prob: 0.5
    min_snr_db: 0
    max_snr_db: 15
  speed:
    prob: 0.5
    sr: ${model.sample_rate}
    resample_type: kaiser_fast
    min_speed_rate: 0.95
    max_speed_rate: 1.05

manifest_filepath: ???
sample_rate: ${model.sample_rate}
labels: null
batch_size: 128
shuffle: false



You will often notice that some configs have ??? in place of paths. This is used as a placeholder so that the user can change the value at a later time.

Let's add the paths to the manifests to the config above
Also, since an4 dataset doesn't have a test set of the same speakers used in training, we will use validation manifest as test manifest for demonstration purposes

In [12]:
config.model.train_ds.manifest_filepath = train_manifest
config.model.validation_ds.manifest_filepath = validation_manifest

Note: Since we are training speaker embedding extractor model for verification we do not add test_ds dataset. To include it add it to config and replace manifest file as 
`config.model.test_ds.manifest_filepath = test_manifest`

Also as we are training on an4 dataset, there are 74 speaker labels in training, and we need to set this in the decoder config

In [13]:
config.model.decoder.num_classes = 74

## Building the PyTorch Lightning Trainer
NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Let us first instantiate a Trainer object!

In [14]:
import torch
import pytorch_lightning as pl

In [15]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

Trainer config - 

devices: 1
max_epochs: 250
max_steps: -1
num_nodes: 1
accelerator: gpu
strategy: ddp
accumulate_grad_batches: 1
deterministic: false
enable_checkpointing: false
logger: false
log_every_n_steps: 1
val_check_interval: 1.0
gradient_clip_val: 1.0



In [16]:
# Let us modify some trainer configs for this demo
# Checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator

# Reduces maximum number of epochs to 5 for quick demonstration
config.trainer.max_epochs = 10

# Remove distributed training flags
config.trainer.strategy = 'auto'

# Remove augmentations
config.model.train_ds.augmentor=None

In [17]:
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import Callback

class MyPrintingCallback(Callback):
    
    def __init__(self):
        super().__init__
        
    def on_train_epoch_end(self, trainer: Trainer, pl_module: LightningModule) -> None:
        print("embeddings")
        return super().on_train_epoch_end(trainer, pl_module)
        
    # def on_init_start(self, trainer):
    #     print("Starting to init trainer!")

    # def on_init_end(self, trainer):
    #     print("trainer is init now")

    # def on_train_end(self, trainer, pl_module):
    #     print("do something when training ends")
        


In [18]:
trainer = pl.Trainer(**config.trainer, callbacks=[MyPrintingCallback()])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


## Setting up a NeMo Experiment
NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it !

In [19]:
from nemo.utils.exp_manager import exp_manager
log_dir = exp_manager(trainer, config.get("exp_manager", None))
# The log_dir provides a path to the current logging directory for easy access
print(log_dir)

[NeMo I 2024-03-11 08:21:57 exp_manager:396] Experiments will be logged at /home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57
[NeMo I 2024-03-11 08:21:57 exp_manager:842] TensorboardLogger has been set up
/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57


## Building the TitaNet Model
TitaNet is a speaker embedding extractor model that can be used for speaker identification tasks - it generates one label for the entire provided audio stream. Therefore we encapsulate it inside the EncDecSpeakerLabelModel as follows.

In [20]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel(cfg=config.model, trainer=trainer)

[NeMo I 2024-03-11 08:21:57 collections:445] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-03-11 08:21:57 collections:446] Dataset loaded with 853 items, total duration of  0.63 hours.
[NeMo I 2024-03-11 08:21:57 collections:448] # 853 files loaded accounting to # 74 labels


[NeMo W 2024-03-11 08:21:57 label_models:189] Total number of 74 found in all the manifest files.


[NeMo I 2024-03-11 08:21:57 collections:445] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-03-11 08:21:57 collections:446] Dataset loaded with 853 items, total duration of  0.63 hours.
[NeMo I 2024-03-11 08:21:57 collections:448] # 853 files loaded accounting to # 74 labels
[NeMo I 2024-03-11 08:21:57 collections:445] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-03-11 08:21:57 collections:446] Dataset loaded with 95 items, total duration of  0.07 hours.
[NeMo I 2024-03-11 08:21:57 collections:448] # 95 files loaded accounting to # 74 labels
[NeMo I 2024-03-11 08:21:57 features:289] PADDING: 16


Before we begin training, let us first create a Tensorboard visualization to monitor progress

In [21]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

To use tensorboard, please use this notebook in a Google Colab environment.


As any NeMo model is inherently a PyTorch Lightning Model, it can easily be trained in a single line - trainer.fit(model)!
Below we see that the model begins to get modest scores on the validation set after just 5 epochs of training

In [22]:
trainer.fit(speaker_model)

You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-03-11 08:21:58 modelPT:723] Optimizer config = SGD (
    Parameter Group 0
        dampening: 0
        differentiable: False
        foreach: None
        lr: 0.08
        maximize: False
        momentum: 0
        nesterov: False
        weight_decay: 0.0002
    )
[NeMo I 2024-03-11 08:21:58 lr_scheduler:915] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f3d95056e90>" 
    will be used during training (effective maximum steps = 140) - 
    Parameters : 
    (warmup_ratio: 0.1
    min_lr: 0.0001
    max_steps: 140
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | loss              | AngularSoftmaxLoss                | 0     
1 | eval_loss         | AngularSoftmaxLoss                | 0     
2 | _accuracy         | TopKClassificationAccuracy        | 0     
3 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
4 | encoder           | ECAPAEncoder                      | 18.1 M
5 | decoder           | SpeakerDecoder                    | 2.8 M 
6 | _macro_accuracy   | MulticlassAccuracy                | 0     
7 | spec_augmentation | SpectrogramAugmentation           | 0     
------------------------------------------------------------------------
20.9 M    Trainable params
0         Non-trainable params
20.9 M    Total params
83.725    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo W 2024-03-11 08:21:58 nemo_logging:349] /home/eyal/git/hakol/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    
[NeMo W 2024-03-11 08:21:59 nemo_logging:349] /home/eyal/git/hakol/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


...

Epoch 8, global step 126: 'val_loss' reached 6.10824 (best 6.10824), saving model to '/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57/checkpoints/ECAPA_TDNN--val_loss=6.1082-epoch=8.ckpt' as top 3


embeddings


Validation: 0it [00:00, ?it/s]

Epoch 9, global step 140: 'val_loss' reached 6.11986 (best 6.10824), saving model to '/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57/checkpoints/ECAPA_TDNN--val_loss=6.1199-epoch=9.ckpt' as top 3
`Trainer.fit` stopped: `max_epochs=10` reached.


embeddings


This config is not suited and designed for an4 so you may observe unstable val_loss

If you have a test manifest file, we can easily compute test accuracy by running
<pre><code>trainer.test(speaker_model, ckpt_path=None)
</code></pre>


## For Faster Training
We can dramatically improve the time taken to train this model by using Multi GPU training along with Mixed Precision.

### Trainer with a distributed backend:
<pre><code>trainer = Trainer(devices=2, num_nodes=2, accelerator='gpu', strategy='auto')
</code></pre>

### Mixed precision:
<pre><code>trainer = Trainer(amp_level='O1', precision=16)
</code></pre>

Of course, you can combine these flags as well.

## Saving/Restoring a checkpoint
There are multiple ways to save and load models in NeMo. Since all NeMo models are inherently Lightning Modules, we can use the standard way that PyTorch Lightning saves and restores models.

NeMo also provides a more advanced model save/restore format, which encapsulates all the parts of the model that are required to restore that model for immediate use.

In this example, we will explore both ways of saving and restoring models, but we will focus on the PyTorch Lightning method.

## Saving and Restoring via PyTorch Lightning Checkpoints
When using NeMo for training, it is advisable to utilize the exp_manager framework. It is tasked with handling checkpointing and logging (Tensorboard as well as WandB optionally!), as well as dealing with multi-node and multi-GPU logging.

Since we utilized the exp_manager framework above, we have access to the directory where the checkpoints exist.

exp_manager with the default settings will save multiple checkpoints for us -

1) A few checkpoints from certain steps of training. They will have --val_loss= tags

2) Checkpoints at the last epoch of training are denoted by --last.

3) If the model finishes training, it will also have a --last checkpoint.

In [23]:
# Let us list all the checkpoints we have
checkpoint_dir = os.path.join(log_dir, 'checkpoints')
checkpoint_paths = list(glob.glob(os.path.join(checkpoint_dir, "*.ckpt")))
checkpoint_paths

['/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57/checkpoints/ECAPA_TDNN--val_loss=6.1984-epoch=7.ckpt',
 '/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57/checkpoints/ECAPA_TDNN--val_loss=6.1199-epoch=10-last.ckpt',
 '/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57/checkpoints/ECAPA_TDNN--val_loss=6.1082-epoch=8.ckpt',
 '/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57/checkpoints/ECAPA_TDNN--val_loss=6.1199-epoch=9.ckpt']

In [24]:
final_checkpoint = list(filter(lambda x: "-last.ckpt" in x, checkpoint_paths))[0]
print(final_checkpoint)

/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57/checkpoints/ECAPA_TDNN--val_loss=6.1199-epoch=10-last.ckpt



## Restoring from a PyTorch Lightning checkpoint
To restore a model using the LightningModule.load_from_checkpoint() class method.

In [25]:
restored_model = nemo_asr.models.EncDecSpeakerLabelModel.load_from_checkpoint(final_checkpoint)

[NeMo W 2024-03-11 08:23:02 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /home/eyal/git/hakol/notebooks/data/an4/wav/an4_clstk/train.json
    sample_rate: 16000
    labels:
    - fash
    - fbbh
    - fclc
    - fejs
    - ffmm
    - fjam
    - fjdn
    - fjmd
    - fkai
    - fkdo
    - flmm2
    - flrp
    - fmjc
    - fmjd
    - fnsv
    - fplp
    - fsaf2
    - fsrb
    - ftal
    - ftmj
    - fwxs
    - mblb
    - mblw
    - mbmg
    - mcel
    - mcen
    - mcfl
    - mcrt
    - mcsc
    - mdcs
    - mdcs2
    - mdmc
    - mdxn
    - mdxs
    - meab
    - meht
    - mema
    - mewl
    - mfaa
    - mgah
    - mjbh
    - mjda
    - mjdr
    - mjes
    - mjgk
    - mjhp
    - mjjs2
    - mkdb
    - mkem
    - mmaf
    - mmal
    - mmap
    - mmdg
    - mmkw
    - mmsh
    - mmtm
    - mnfe
    - mnjl
    -

[NeMo I 2024-03-11 08:23:02 features:289] PADDING: 16


# Finetuning
Since we don't have any new manifest file to finetune, I will demonstrate here by using the test manifest file we created earlier. 
an4 test dataset has a different set of speakers from the train set (total number: 10). And as we didn't split this dataset for validation I will use the same for validation. 

There are a couple of ways we can finetune a speaker recognition model. 
1. Finetuning using a pretrained model published on NGC. 
2. Finetuning from a PTL checkpoint. 

Since finetuning from a large pretrained model is more common, I shall use it to demonstrate finetuning procedure. In order to make finetuning step independent from training from scratch, we use another config. Here we shall use `titanet-finetune.yaml` config, that is created to show finetuning on pretrained titanet-large model. 

Note: You may use [finetune-script](https://github.com/NVIDIA/NeMo/blob/main/examples/speaker_tasks/recognition/speaker_reco_finetune.py) to launch training in the command line. Following is just a demonstration of the script

In [26]:
!wget -P conf https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/speaker_tasks/recognition/conf/titanet-finetune.yaml
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/titanet-finetune.yaml')
finetune_config = OmegaConf.load(MODEL_CONFIG)
print(OmegaConf.to_yaml(finetune_config))

--2024-03-11 08:23:02--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/recognition/conf/titanet-finetune.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4507 (4.4K) [text/plain]
Saving to: ‘conf/titanet-finetune.yaml’

titanet-finetune.ya 100%[===================>]   4.40K  --.-KB/s    in 0s      

2024-03-11 08:23:03 (29.4 MB/s) - ‘conf/titanet-finetune.yaml’ saved [4507/4507]

name: TitaNet-Finetune
sample_rate: 16000
init_from_pretrained_model:
  speaker_tasks:
    name: titanet_large
    include:
    - preprocessor
    - encoder
    exclude:
    - decoder.final
model:
  train_ds:
    manifest_filepath: ???
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_fi

For step 2, if one would like to finetune from a PTL checkpoint, `init_from_pretrained_model` in config should be replaced with `init_from_nemo_model` and need to provide the path to checkpoint. 

In [27]:
test_manifest = os.path.join(data_dir,'an4/wav/an4test_clstk/test.json')
finetune_config.model.train_ds.manifest_filepath = test_manifest
finetune_config.model.validation_ds.manifest_filepath = test_manifest
finetune_config.model.decoder.num_classes = 10

So we have set up the data and changed the decoder required for finetune, we now just need to create a trainer and start training with a smaller learning rate for fewer epochs

In [28]:
# Setup the new trainer object
# Let us modify some trainer configs for this demo
# Checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'

trainer_config = OmegaConf.create(dict(
    devices=1,
    accelerator=accelerator,
    max_epochs=5,
    max_steps=-1,  # computed at runtime if not set
    num_nodes=1,
    accumulate_grad_batches=1,
    enable_checkpointing=False,  # Provided by exp_manager
    logger=False,  # Provided by exp_manager
    log_every_n_steps=1,  # Interval of logging.
    val_check_interval=1.0,  # Set to 0.25 to check 4 times per epoch, or an int for number of iterations
))
print(OmegaConf.to_yaml(trainer_config))

devices: 1
accelerator: gpu
max_epochs: 5
max_steps: -1
num_nodes: 1
accumulate_grad_batches: 1
enable_checkpointing: false
logger: false
log_every_n_steps: 1
val_check_interval: 1.0



In [29]:
trainer_finetune = pl.Trainer(**trainer_config)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


## Setting the trainer to the restored model
Setting the trainer to the restored model

In [30]:
log_dir_finetune = exp_manager(trainer_finetune, config.get("exp_manager", None))
print(log_dir_finetune)

[NeMo I 2024-03-11 08:23:03 exp_manager:396] Experiments will be logged at /home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57
[NeMo I 2024-03-11 08:23:03 exp_manager:842] TensorboardLogger has been set up
/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57


## Fine-tune training step

When fine-tuning on a truly new dataset, we will not see such a dramatic improvement in performance. However, it should still converge a little faster than if it was trained from scratch.

In [31]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel(cfg=finetune_config.model, trainer=trainer_finetune)
speaker_model.maybe_init_from_pretrained_checkpoint(finetune_config)

[NeMo I 2024-03-11 08:23:04 collections:445] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-03-11 08:23:04 collections:446] Dataset loaded with 130 items, total duration of  0.10 hours.
[NeMo I 2024-03-11 08:23:04 collections:448] # 130 files loaded accounting to # 10 labels


[NeMo W 2024-03-11 08:23:04 label_models:189] Total number of 10 found in all the manifest files.


[NeMo I 2024-03-11 08:23:04 collections:445] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-03-11 08:23:04 collections:446] Dataset loaded with 130 items, total duration of  0.10 hours.
[NeMo I 2024-03-11 08:23:04 collections:448] # 130 files loaded accounting to # 10 labels
[NeMo I 2024-03-11 08:23:04 collections:445] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-03-11 08:23:04 collections:446] Dataset loaded with 130 items, total duration of  0.10 hours.
[NeMo I 2024-03-11 08:23:04 collections:448] # 130 files loaded accounting to # 10 labels
[NeMo I 2024-03-11 08:23:04 features:289] PADDING: 16
[NeMo I 2024-03-11 08:23:04 cloud:58] Found existing object /home/eyal/.cache/torch/NeMo/NeMo_1.23.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2024-03-11 08:23:04 cloud:64] Re-using file from: /home/eyal/.cache/torch/NeMo/NeMo_1.23.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2024-03-11 08:23:04 com

[NeMo W 2024-03-11 08:23:04 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2024-03-11 08:23:04 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2024-03-11 08:23:04 features:289] PADDING: 16
[NeMo I 2024-03-11 08:23:05 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/eyal/.cache/torch/NeMo/NeMo_1.23.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2024-03-11 08:23:05 modelPT:1145] Model checkpoint partially restored from pretrained checkpoint with name `titanet_large`
[NeMo I 2024-03-11 08:23:05 modelPT:1147] The following parameters were excluded when loading from pretrained checkpoint with name `titanet_large` : ['decoder.final.weight']
[NeMo I 2024-03-11 08:23:05 modelPT:1150] Make sure that this is what you wanted!


In the config, we keep weights of preprocessor and encoder, and attach a new decoder as mentioned in above section to match num of classes of new data

In [32]:
## Fine-tuning for 5 epochs¶
trainer_finetune.fit(speaker_model)

[NeMo W 2024-03-11 08:23:05 nemo_logging:349] /home/eyal/git/hakol/.venv/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57/checkpoints exists and is not empty.
      rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
    
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-03-11 08:23:05 modelPT:723] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.0001
        maximize: False
        weight_decay: 0.0002
    
    Parameter Group 1
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.001
        maximize: False
        weight_decay: 0.0002
    )
[NeMo I 2024-03-11 08:23:05 lr_scheduler:915] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f3d70164d30>" 
    will be used during training (effective maximum steps = 15) - 
    Parameters : 
    (warmup_ratio: 0.1
    min_lr: 0.0
    max_steps: 15
    )



  | Name            | Type                              | Params
----------------------------------------------------------------------
0 | loss            | AngularSoftmaxLoss                | 0     
1 | eval_loss       | AngularSoftmaxLoss                | 0     
2 | _accuracy       | TopKClassificationAccuracy        | 0     
3 | preprocessor    | AudioToMelSpectrogramPreprocessor | 0     
4 | encoder         | ConvASREncoder                    | 19.4 M
5 | decoder         | SpeakerDecoder                    | 2.8 M 
6 | _macro_accuracy | MulticlassAccuracy                | 0     
----------------------------------------------------------------------
22.1 M    Trainable params
0         Non-trainable params
22.1 M    Total params
88.497    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo W 2024-03-11 08:23:05 nemo_logging:349] /home/eyal/git/hakol/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    
[NeMo W 2024-03-11 08:23:05 nemo_logging:349] /home/eyal/git/hakol/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


...

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 3: 'val_loss' reached 11.26189 (best 11.26189), saving model to '/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57/checkpoints/ECAPA_TDNN--val_loss=11.2619-epoch=0.ckpt' as top 3


Validation: 0it [00:00, ?it/s]

Epoch 1, global step 6: 'val_loss' reached 4.12678 (best 4.12678), saving model to '/home/eyal/git/hakol/notebooks/nemo_experiments/ECAPA_TDNN/2024-03-11_08-21-57/checkpoints/ECAPA_TDNN--val_loss=4.1268-epoch=1.ckpt' as top 3


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.44 GiB. GPU 0 has a total capacity of 47.53 GiB of which 1.38 GiB is free. Process 280006 has 15.73 GiB memory in use. Process 324255 has 14.83 GiB memory in use. Including non-PyTorch memory, this process has 15.41 GiB memory in use. Of the allocated memory 11.75 GiB is allocated by PyTorch, and 3.31 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Tip: Add more data augmentation and dropout while finetuning on your data

# Saving .nemo file
Now we can save the whole config and model parameters in a single .nemo and we can anytime restore from it

In [ ]:
saved_model = 'titanet-small-finetune.nemo'
restored_model.save_to(os.path.join(log_dir_finetune, '..',saved_model))

In [ ]:
!ls {log_dir_finetune}/..

In [ ]:
# restore from a save model
restored_model = nemo_asr.models.EncDecSpeakerLabelModel.restore_from(os.path.join(log_dir_finetune, '..', saved_model))


# Speaker Verification

Training for a speaker verification model is almost the same as the speaker recognition model with a change in the loss function. Angular Loss is a better function to train for a speaker verification model as the model is trained in an end-to-end manner with loss optimizing for embeddings cluster to be far from each other for different speaker by maximizing the angle between these clusters

To train for verification we just need to toggle `angular` flag in `config.model.decoder.params.angular = True` else set it to `False` to train with cross-entropy loss for identification purposes. 
Once we set this, the loss will be changed to angular loss and we can follow the above steps to the model.
Note the scale and margin values to be set for the loss function are present at `config.model.loss.scale` and `config.model.loss.margin`

## Extract Speaker Embeddings
Once you have a trained model or use one of our pretrained nemo checkpoints to get speaker embeddings for any speaker.

To demonstrate this we shall use `nemo_asr.models.EncDecSpeakerLabelModel` with say 5 audio_samples from our dev manifest set. This model is specifically for inference purposes to extract embeddings from a trained `.nemo` model

In [ ]:
verification_model = nemo_asr.models.EncDecSpeakerLabelModel.restore_from(os.path.join(log_dir_finetune, '..', saved_model))

Now, we need to pass the necessary manifest_filepath and params to set up the data loader for extracting embeddings

In [ ]:
!head -5 {validation_manifest} > embeddings_manifest.json

In [ ]:
config.model.train_ds

In [ ]:
from nemo.collections.asr.parts.utils.speaker_utils import embedding_normalize
from tqdm import  tqdm
try:
    from torch.cuda.amp import autocast
except ImportError:
    from contextlib import contextmanager

    @contextmanager
    def autocast(enabled=None):
        yield
import numpy as np
import json
import pickle as pkl

In [ ]:
def get_embeddings(speaker_model, manifest_file, batch_size=1, embedding_dir='./', device='cuda'):
    test_config = OmegaConf.create(
        dict(
            manifest_filepath=manifest_file,
            sample_rate=16000,
            labels=None,
            batch_size=batch_size,
            shuffle=False,
            time_length=20,
        )
    )

    speaker_model.setup_test_data(test_config)
    speaker_model = speaker_model.to(device)
    speaker_model.eval()

    all_embs=[]
    out_embeddings = {}
           
    for test_batch in tqdm(speaker_model.test_dataloader()):
        test_batch = [x.to(device) for x in test_batch]
        audio_signal, audio_signal_len, labels, slices = test_batch
        with autocast():
            _, embs = speaker_model.forward(input_signal=audio_signal, input_signal_length=audio_signal_len)
            emb_shape = embs.shape[-1]
            embs = embs.view(-1, emb_shape)
            all_embs.extend(embs.cpu().detach().numpy())
        del test_batch

    all_embs = np.asarray(all_embs)
    all_embs = embedding_normalize(all_embs)
    with open(manifest_file, 'r') as manifest:
        for i, line in enumerate(manifest.readlines()):
            line = line.strip()
            dic = json.loads(line)
            uniq_name = '@'.join(dic['audio_filepath'].split('/')[-3:])
            out_embeddings[uniq_name] = all_embs[i]

    embedding_dir = os.path.join(embedding_dir, 'embeddings')
    if not os.path.exists(embedding_dir):
        os.makedirs(embedding_dir, exist_ok=True)

    prefix = manifest_file.split('/')[-1].rsplit('.', 1)[-2]

    name = os.path.join(embedding_dir, prefix)
    embeddings_file = name + '_embeddings.pkl'
    pkl.dump(out_embeddings, open(embeddings_file, 'wb'))
    print("Saved embedding files to {}".format(embedding_dir))

In [ ]:
manifest_filepath = os.path.join(NEMO_ROOT,'embeddings_manifest.json')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
get_embeddings(verification_model, manifest_filepath, batch_size=64,embedding_dir='./', device=device)

Embeddings are stored in dict structure with key-value pair, key being uniq_name generated based on audio_filepath of the sample present in manifest_file in `embedding_dir`

In [ ]:
!ls ./embeddings/